In [3]:
from collections import OrderedDict
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import sampler
import os
import torchvision.datasets as dset
import torchvision.transforms as T
import pandas as pd
# for plotting
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

from trainer_functions.cifartrainer import evaluate_cifar, train_cifar, build_cifar10
from models.ResNet import ResNetCifar as ResNet
from trainer_functions.cifarCloader import CIFARCDataset

In [4]:
# empty dataframe
df = pd.DataFrame(columns=['corruption', 'accuracy', "lr", "state", "n_subset"])

curruption_list = ["brightness", "contrast", "defocus_blur", "elastic_transform", "fog", "frost", "gaussian_blur", "gaussian_noise", "glass_blur",
 "impulse_noise", "jpeg_compression", "motion_blur", "pixelate", "saturate", "shot_noise", "snow", "spatter", "speckle_noise", "zoom_blur"]

n_subset = 2000
lrs = [1e-3, 1e-4, 1e-5]

## writing fine tuning code here

In [5]:
for corruption in curruption_list:
    for lr in lrs:
        cifarc = CIFARCDataset(corruption=corruption)
        ccdataloader = DataLoader(cifarc, batch_size=64, shuffle=True)

        cctrain = torch.utils.data.Subset(cifarc, range(0, n_subset))
        ccvalid = torch.utils.data.Subset(cifarc, range(6000, 8000))
        cctest = torch.utils.data.Subset(cifarc, range(8000, 10000))

        cctrainload = DataLoader(cctrain, batch_size=64, shuffle=True)
        ccvalidload = DataLoader(ccvalid, batch_size=64, shuffle=True)
        cctestload = DataLoader(cctest, batch_size=64, shuffle=True)


        device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')

        tune_net = build_cifar10(device)

        optimizer = optim.Adam(tune_net.layer2.parameters(), lr=lr, weight_decay=0.0001)

        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

        checkpoint = torch.load('checkpoints/ckpt.pth')
        tune_net.load_state_dict(checkpoint['net'])


        acc = train_cifar(tune_net, cctrainload, ccvalidload, optimizer, scheduler, device=device, epochs=15)

        # adding results to dataframe
        dict = {}
        dict['corruption'] = corruption
        dict['accuracy'] = acc
        dict['lr'] = lr
        dict['state'] = "all layers"
        dict['n_subset'] = n_subset
        df_temp = pd.DataFrame(dict, index=[0])
        df = pd.concat([df, df_temp])

        

Building model...
Epoch 0, loss = 0.6324, lr 0.00100000
Got 1658 / 2000 correct (82.90)

Epoch 1, loss = 0.3378, lr 0.00100000
Got 1683 / 2000 correct (84.15)

Epoch 2, loss = 0.1469, lr 0.00100000
Got 1669 / 2000 correct (83.45)

Epoch 3, loss = 0.0917, lr 0.00100000
Got 1663 / 2000 correct (83.15)

Epoch 4, loss = 0.0498, lr 0.00100000
Got 1698 / 2000 correct (84.90)

Epoch 5, loss = 0.0169, lr 0.00100000
Got 1688 / 2000 correct (84.40)

Epoch 6, loss = 0.0036, lr 0.00100000
Got 1690 / 2000 correct (84.50)

Epoch 7, loss = 0.0109, lr 0.00100000
Got 1693 / 2000 correct (84.65)

Epoch 8, loss = 0.0083, lr 0.00100000
Got 1698 / 2000 correct (84.90)

Epoch 9, loss = 0.0041, lr 0.00100000
Got 1695 / 2000 correct (84.75)

Epoch 10, loss = 0.0067, lr 0.00100000
Got 1693 / 2000 correct (84.65)

Epoch 11, loss = 0.0076, lr 0.00100000
Got 1699 / 2000 correct (84.95)

Epoch 12, loss = 0.0045, lr 0.00100000
Got 1700 / 2000 correct (85.00)

Epoch 13, loss = 0.0077, lr 0.00100000
Got 1700 / 2000 c

In [ ]:
df.to_csv("cifarC_layer2.csv")

In [ ]:
tune_net

ResNetCifar(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (bn1): GroupNorm(8, 16, eps=1e-05, affine=True)
      (relu1): ReLU(inplace=True)
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): GroupNorm(8, 16, eps=1e-05, affine=True)
      (relu2): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (1): BasicBlock(
      (bn1): GroupNorm(8, 16, eps=1e-05, affine=True)
      (relu1): ReLU(inplace=True)
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): GroupNorm(8, 16, eps=1e-05, affine=True)
      (relu2): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (2): BasicBlock(
      (bn1): GroupNorm(8, 16, eps=1e-05, affine=True)
      (relu1): ReLU(in